# Manufacturing Robot Predictive Maintenance Workshop

A real-time data streaming and visualization system for predictive maintenance of industrial manufacturing robots.

## Cell 1: Import Libraries and Setup

This cell imports all required libraries and configures matplotlib to use the Qt backend for separate plot windows.

In [ ]:
import sys
import os

# Add src directory to Python path (notebook is in notebook/, src is at ../src/)
project_root = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == 'notebook' else os.getcwd()
src_dir = os.path.join(project_root, 'src')
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

print(f"Project root: {project_root}")
print(f"Source directory: {src_dir}")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from datetime import datetime
import warnings
import importlib
warnings.filterwarnings('ignore')

# Use Qt or Tk backend for separate windows (NOT inline)
# This ensures plots open in new windows
%matplotlib qt

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Import StreamingSimulator
import StreamingSimulator
importlib.reload(StreamingSimulator)
from StreamingSimulator import StreamingSimulator

print("Libraries imported successfully")
print(f"Matplotlib backend: {plt.get_backend()}")
print("Plots will appear in SEPARATE WINDOWS (not inline)")

## Cell 2: Database Configuration

Configure your Neon.tech database connection.
Sign up at https://neon.tech/ and get your connection details.

In [ ]:
# Database configuration
db_config = {
    'host': 'ep-polished-snow-ahx3qiod-pooler.c-3.us-east-1.aws.neon.tech',
    'database': 'neondb',
    'user': 'neondb_owner',
    'password': 'npg_JlIENr3i4AbL',
    'port': 5432,
    'sslmode': 'require'
}

# CSV path (data/ is at project root, notebook runs from notebook/)
csv_path = os.path.join(project_root, 'data', 'RMBR4-2_export_test.csv')

# Verify the file exists
if os.path.exists(csv_path):
    print("Configuration loaded")
    print(f"CSV Path: {csv_path}")
    print(f"File exists: Yes")
else:
    # Try alternative paths
    alt_path = '../data/RMBR4-2_export_test.csv'
    if os.path.exists(alt_path):
        csv_path = alt_path
        print("Configuration loaded (using relative path)")
        print(f"CSV Path: {csv_path}")
        print(f"File exists: Yes")
    else:
        print(f"ERROR: CSV file not found!")
        print(f"Current working directory: {os.getcwd()}")
        print(f"Tried path: {csv_path}")
        print("\nPlease ensure the CSV file is in the 'data' folder")

## STEP 1: Data Collection Mechanism - Streaming Simulation

### Manufacturing Robot Controller Data Streaming System

**Real-world Context:**

In industrial robotics, controllers continuously monitor current draw across multiple axes (motors). Each axis represents a degree of freedom:
- **Axis #1-3**: Primary movement (X, Y, Z positioning)
- **Axis #4-6**: Rotational movements (pitch, roll, yaw)
- **Axis #7-14**: Tool-specific axes (grippers, welders, etc.)

**Current measurements indicate:**
- Motor load and workload
- Mechanical resistance (wear, misalignment)
- Electrical issues (shorts, overheating)
- Operational efficiency

**This simulation:**
1. Loads 39,000+ records from the robot controller CSV
2. Batch loads data to cloud database (Neon.tech PostgreSQL)
3. Streams data 1 point at a time from database for real-time visualization
4. Visualizes 8 critical axes in real-time dashboard
5. Monitors current draw patterns across all 14 axes

**Key Features:**
- **Non-blocking visualization**: GUI remains responsive during streaming
- **Close-to-stop**: Click the plot window's X button to stop streaming
- **Single-point streaming**: True real-time visualization (1 record at a time)
- **Optional GPU acceleration**: Install CuPy for faster processing

In [ ]:
# Initialize the Streaming Simulator
print("Initializing Streaming Simulator...")
print("=" * 80)

ss = StreamingSimulator(csv_path=csv_path, db_config=db_config)

print(f"""
✅ Simulator Initialized Successfully!

Dataset Information:
- Total records loaded: {len(ss.df):,}
- Number of robot axes: {len(ss.axis_columns)}
- Axes detected: {', '.join(ss.axis_columns)}
- Current position: {ss.current_index}
- Database: Connected and table created
- Time span: {ss.df['Time'].max() - ss.df['Time'].min()}
""")

## Cell 3: Stream Data from Database

### Two-Phase Streaming Process:

**Phase 1: Batch Load to Database**
- Loads all CSV records into the PostgreSQL database using optimized batch inserts
- Uses execute_values for maximum throughput (~10,000+ records/second)

**Phase 2: Real-Time Database Streaming (1 Point at a Time)**
- Fetches **exactly 1 record** per database call for true real-time streaming
- Updates visualization after **every single point**
- Uses **non-blocking visualization** to keep GUI responsive
- **Close the plot window** to stop streaming at any time

### Streaming Parameters:
```python
ss.stream_from_db(
    visualize_every=1,    # Update plot every N points (1 = every point)
    plot_style='combined', # 'combined' or 'separate' axes view
    fetch_size=1          # Records per DB fetch (1 = true real-time)
)
```

### Performance Notes:
- Single-point streaming (~10-50 pts/sec) provides smooth real-time visualization
- For faster bulk processing, increase `fetch_size` (e.g., 100) and `visualize_every` (e.g., 50)

### GPU Acceleration (Optional)
Install CuPy for GPU-accelerated processing:
```bash
pip install cupy-cuda12x  # For CUDA 12.x
```

In [ ]:
# Phase 1: Load all CSV data into database using optimized batch method
print("=" * 80)
print("PHASE 1: Loading CSV data into database...")
print("=" * 80)

# Stream ALL data points using batch processing for fast database loading
num_points_to_stream = len(ss.df)  # Stream entire dataset

print(f"Target: Load {num_points_to_stream:,} data points into database")
print("Method: Batch inserts with psycopg2 execute_values")
print()

# Use the optimized batch streaming method
result = ss.streamBatch(
    num_points=num_points_to_stream,
    batch_size=1000,
    visualize_every=0,  # Disable visualization during loading
    plot_style='combined'
)

print("\n" + "=" * 80)
print(f"Database loaded: {ss.current_index:,} data points")
print("=" * 80)

# Reset simulator for database streaming
ss.reset()

# Get total records from database
db_count = ss.get_db_record_count()
print(f"\nRecords in database: {db_count:,}")

print("\n" + "=" * 80)
print("PHASE 2: Streaming from database (1 point at a time)...")
print("=" * 80)
print("\nPlots will open in a SEPARATE WINDOW")
print("Close the plot window to stop streaming")
print("Press Ctrl+C (or interrupt kernel) to stop early\n")

# Check if GPU is available
from StreamingSimulator import GPU_AVAILABLE
if GPU_AVAILABLE:
    print("GPU detected! Using GPU-accelerated streaming...")
    summary = ss.stream_from_db_gpu(visualize_every=1, plot_style='combined', batch_size=1)
else:
    print("Streaming 1 point at a time with real-time visualization...")
    # Stream 1 point at a time for true real-time visualization
    # fetch_size=1 means each database call fetches exactly 1 record
    # visualize_every=1 means update the plot after every single point
    summary = ss.stream_from_db(visualize_every=1, plot_style='combined', fetch_size=1)

print("\n" + "=" * 80)
print("STREAMING COMPLETE")
print("=" * 80)
if summary:
    print(f"Points processed: {summary['points_processed']:,}")
    print(f"Buffer size: {len(ss.data_buffer):,} records")
    print(f"Total time: {summary['elapsed_seconds']:.1f} seconds")
    print(f"Rate: {summary['points_per_second']:.1f} points/second")
    if summary.get('gpu_accelerated'):
        print("Acceleration: GPU (CuPy)")
    if summary.get('stopped_early'):
        print("Status: Stopped early by user")

## STEP 2: Data Source Documentation

### Robot Controller Multi-Axis Current Monitoring Data

**Data Source Overview:**

This dataset contains real-time current measurements from an industrial manufacturing robot controller. The robot operates in a production environment performing precision assembly, welding, or material handling tasks.

**Dataset Characteristics:**
- **Total Records**: 39,000+ sensor readings
- **Collection Period**: Continuous operational monitoring
- **Sampling Frequency**: Variable (approximately 1-5 second intervals)
- **Data Type**: Multi-axis motor current measurements
- **Format**: CSV with tab-separated values

**Data Schema:**

| Field       | Type      | Description                                    | Unit    |
|-------------|-----------|------------------------------------------------|---------|
| Trait       | String    | Measurement type (always "current")            | -       |
| Axis #1-14  | Float     | Current draw for each robot axis               | Amperes |
| Time        | DateTime  | ISO 8601 timestamp of measurement              | -       |

**Axis Configuration:**

- **Axes 1-3**: Primary Linear Motors (X, Y, Z positioning)
  - Control base movement and positioning
  - Typical range: 0-30A
  - Critical for precision placement

- **Axes 4-6**: Rotational Joints (Wrist/Elbow rotation)
  - Enable angular positioning
  - Typical range: 0-15A
  - Important for orientation tasks

- **Axes 7-8**: Tool Control (End effector, gripper)
  - Operate specialized tools
  - Typical range: 0-10A
  - Task-specific current patterns

- **Axes 9-14**: Auxiliary/Reserved
  - Additional DOF for complex robots
  - May be null for simpler configurations
  - Used in 6+ axis robots

**Current Measurement Significance:**

1. **Mechanical Load**: Higher current = higher resistance/load
2. **Wear Detection**: Increasing current over time suggests bearing wear
3. **Misalignment**: Erratic current patterns indicate mechanical issues
4. **Efficiency**: Consistent current = healthy motor operation
5. **Anomalies**: Spikes/drops signal potential failures

**Use Case Applications:**
- Predictive maintenance scheduling
- Motor health monitoring
- Energy consumption optimization
- Workload balancing across axes
- Failure prevention (before catastrophic breakdown)
- Quality assurance (detect anomalies during production)

**Data Quality Considerations:**
- Some axes may have NULL values (not all robots use all 14 axes)
- Timestamps may have irregular intervals due to event-driven logging
- Current values of 0 may indicate idle state or sensor issues
- Outliers may represent legitimate peak loads or sensor errors

---

**Query Test 1:** "What current patterns in specific axes indicate impending motor failure or mechanical issues?"

**Query Test 2:** "How can we correlate current anomalies across multiple axes to predict system-wide maintenance needs?"

In [ ]:
# Display comprehensive dataset information
print("=" * 80)
print("DATASET DETAILED ANALYSIS")
print("=" * 80)

print("\n1. DATASET STRUCTURE")
print("-" * 80)
print(ss.df.info())

print("\n2. FIRST 10 RECORDS (Raw Data)")
print("-" * 80)
print(ss.df.head(10).to_string())

print("\n3. STATISTICAL SUMMARY (All Axes)")
print("-" * 80)
print(ss.df[ss.axis_columns].describe().to_string())

print("\n4. DATA COMPLETENESS CHECK")
print("-" * 80)
for axis in ss.axis_columns:
    null_count = ss.df[axis].isna().sum()
    null_pct = (null_count / len(ss.df)) * 100
    print(f"{axis}: {null_count:,} nulls ({null_pct:.2f}%)")

print("\n5. TIME DISTRIBUTION")
print("-" * 80)
time_diff = ss.df['Time'].diff().describe()
print(f"Average interval: {time_diff['mean']}")
print(f"Min interval: {time_diff['min']}")
print(f"Max interval: {time_diff['max']}")

## Cell 4: Comprehensive Data Visualization

Static visualizations of the complete dataset including:
- Current distribution histograms for all 14 axes
- Time series plots for primary 8 axes
- Correlation heatmap between axes

In [ ]:
print("=" * 80)
print("GENERATING COMPREHENSIVE VISUALIZATIONS")
print("=" * 80)

# Visualization 1: Current Distribution Across All Axes
fig, axes = plt.subplots(3, 5, figsize=(20, 12))
fig.suptitle('Current Distribution Across All 14 Robot Axes', 
             fontsize=16, fontweight='bold')

for idx, axis in enumerate(ss.axis_columns):
    row = idx // 5
    col = idx % 5
    ax = axes[row, col]
    
    data = ss.df[axis].dropna()
    
    if len(data) > 0:
        ax.hist(data, bins=50, color=f'C{idx}', alpha=0.7, edgecolor='black')
        ax.axvline(data.mean(), color='red', linestyle='--', 
                   linewidth=2, label=f'Mean: {data.mean():.2f}A')
        ax.set_title(f'{axis}', fontsize=10, fontweight='bold')
        ax.set_xlabel('Current (A)', fontsize=8)
        ax.set_ylabel('Frequency', fontsize=8)
        ax.legend(fontsize=7)
        ax.grid(True, alpha=0.3)

axes[2, 4].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Visualization 2: Time Series Overview (First 500 points)
fig, axes = plt.subplots(4, 2, figsize=(18, 14))
fig.suptitle('Current Over Time - Primary Axes (First 500 Data Points)', 
             fontsize=16, fontweight='bold')

sample_data = ss.df.head(500)
primary_axes = ss.axis_columns[:8]

for idx, axis in enumerate(primary_axes):
    row = idx // 2
    col = idx % 2
    ax = axes[row, col]
    
    data = sample_data[axis].dropna()
    ax.plot(data.index, data.values, linewidth=1.5, alpha=0.8)
    ax.set_title(f'{axis} Current Trend', fontsize=11, fontweight='bold')
    ax.set_xlabel('Reading Index', fontsize=9)
    ax.set_ylabel('Current (A)', fontsize=9)
    ax.grid(True, alpha=0.3, linestyle=':')
    ax.axhline(data.mean(), color='red', linestyle='--', 
              linewidth=1, alpha=0.5, label=f'Mean: {data.mean():.2f}A')
    ax.legend(fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
# Visualization 3: Correlation Heatmap
print("\n📊 Generating correlation analysis...")
correlation_matrix = ss.df[ss.axis_columns].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Current Correlation Between Robot Axes', 
          fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("""
Correlation Insights:
- High positive correlation (>0.7): Axes moving together (coordinated motion)
- High negative correlation (<-0.7): Compensating movements
- Low correlation (~0): Independent axis operations
""")

## STEP 3: Anomaly Detection and Analysis

### Statistical Anomaly Detection for Multi-Axis Robot Systems

**Methodology:**

We employ statistical process control (SPC) methods to identify anomalies:
- Calculate mean (μ) and standard deviation (σ) for each axis
- Define control limits: μ ± 2.5σ (captures 98.76% of normal data)
- Flag data points outside control limits as anomalies

**Anomaly Categories:**

1. **Positive Spikes**: Current exceeds upper threshold
   - Indicates: Overload, jamming, mechanical resistance
   
2. **Negative Drops**: Current below lower threshold
   - Indicates: Loss of load, disconnection, calibration drift
   
3. **High Variability**: Frequent threshold crossings
   - Indicates: Unstable operation, vibration, loose components

**GPU Acceleration (Optional):**

For large datasets, enable GPU-accelerated anomaly detection:
```python
anomalies = ss.detect_anomalies(threshold_multiplier=2.5, use_gpu=True)
```
Requires CuPy: `pip install cupy-cuda12x`

In [ ]:
print("=" * 80)
print("ANOMALY DETECTION ANALYSIS")
print("=" * 80)

anomalies = ss.detect_anomalies(threshold_multiplier=2.5)

if not anomalies:
    print("\n✅ NO ANOMALIES DETECTED in current data stream")
    print("All axes operating within normal parameters.")
else:
    print(f"\n⚠️  ANOMALIES DETECTED: {len(anomalies)} axes affected\n")
    
    for axis, details in anomalies.items():
        print(f"\n{'='*70}")
        print(f"🔍 {axis} - {details['severity']} SEVERITY")
        print(f"{'='*70}")
        print(f"   Anomaly Count: {len(details['indices'])} occurrences")
        print(f"   Mean Current: {details['mean']:.3f} A")
        print(f"   Std Deviation: {details['std']:.3f} A")
        print(f"   Upper Threshold: {details['threshold_upper']:.3f} A")
        print(f"   Lower Threshold: {details['threshold_lower']:.3f} A")
        print(f"   Anomalous Readings:")
        
        for i, (idx, val) in enumerate(zip(details['indices'][:5], 
                                           details['values'][:5])):
            print(f"      [{i+1}] Index {idx}: {val:.3f} A " + 
                  f"({'↑' if val > details['threshold_upper'] else '↓'} " +
                  f"{abs(val - details['mean']):.2f}A from mean)")
        
        if len(details['indices']) > 5:
            print(f"      ... and {len(details['indices']) - 5} more")

## Cell 5: Axis Health Analysis

Analyzes the health status of each robot axis based on:
- Mean current draw
- Standard deviation
- Coefficient of variation (CV)
- Maximum current

**Health Status Levels:**
- **NORMAL** (Green): CV < 30%, max current < 30A
- **CAUTION** (Orange): Max current > 30A
- **WARNING** (Yellow): CV > 30%
- **CRITICAL** (Red): CV > 50%

In [ ]:
print("=" * 80)
print("ROBOT AXIS HEALTH ASSESSMENT")
print("=" * 80)

health_report = ss.analyze_axis_health()

if health_report:
    print(f"\nAnalyzing {len(health_report)} active axes...\n")
    
    for axis, metrics in sorted(health_report.items()):
        print(f"{metrics['color']} {axis}: {metrics['status']}")
        print(f"   Mean Current: {metrics['mean_current']:.3f} A")
        print(f"   Std Deviation: {metrics['std_current']:.3f} A")
        print(f"   Max Current: {metrics['max_current']:.3f} A")
        print(f"   Variability (CV): {metrics['coefficient_of_variation']:.1f}%")
        print(f"   Data Points: {metrics['data_points']}")
        print()